<div style="background-color:#000;"><img src="pqn.png"></img></div>

## Library installation
Install the libraries we use so the notebook runs anywhere without manual setup.

In [ ]:
!pip install yfinance pandas numpy matplotlib seaborn statsmodels

These are standard pip packages that cover data access (yfinance), manipulation (pandas, numpy), testing (statsmodels), and plotting (matplotlib, seaborn). If you use conda, you can install the same names via conda-forge to avoid binary build issues on some systems.

## Imports and setup
We use matplotlib and seaborn for plotting, numpy and pandas for array/DataFrame work, yfinance to fetch historical prices, and statsmodels’ coint for the Engle–Granger cointegration test.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import yfinance as yf
from statsmodels.tsa.stattools import coint

These imports set up a minimal stack that mirrors how professionals prototype stat-arb: get data, test the relationship, and visualize the spread. Keeping the toolset small helps us focus on the mechanics (stationarity and residuals) rather than juggling libraries.

## Define universe and download data
Define a related renewable-energy universe and pull daily closes, then align the panel by dropping dates with missing values.

In [ ]:
tickers = [
    "NEE",
    "FSLR",
    "ENPH",
    "PLUG",
    "BEP",
    "AQN",
    "PBW",
    "FAN",
    "ICLN",
]
start_date = "2014-01-01"
end_date = "2015-01-01"
df = yf.download(
    tickers,
    start=start_date,
    end=end_date,
    auto_adjust=False,
).Close
df = df.dropna()

Starting with related names increases our odds of finding economic links that support a stable residual, rather than spurious correlation. Dropping missing dates ensures the Engle–Granger test compares like-for-like observations, which protects your inference. The short window is for illustration; in practice, use longer histories and time splits to reduce sampling noise.

## Test cointegration and select pairs
Run Engle–Granger tests across all pairs to score stability (p-values) and keep candidates below a permissive threshold.

In [ ]:
n = len(tickers)
score_matrix = np.zeros((n, n))
pvalue_matrix = np.ones((n, n))
pairs = []
for i in range(n):
    for j in range(i + 1, n):
        score, pval, _ = coint(df.iloc[:, i], df.iloc[:, j])
        score_matrix[i, j] = score
        pvalue_matrix[i, j] = pval
        if pval < 0.10:
            pairs.append((tickers[i], tickers[j]))

Cointegration targets a stationary linear combination, which is what makes residual trading coherent when volatility shifts. A 10% screen is deliberately loose so we don’t overfit at this stage; we’ll rely on later standardization and validation to refine signals. In production research, account for multiple testing and re-estimate over rolling windows to keep parameters current.

Identify the strongest candidate (lowest p-value) and extract its two aligned price series for further analysis.

In [ ]:
mask = np.triu(np.ones_like(pvalue_matrix, dtype=bool), k=1)
upper_vals = pvalue_matrix[mask]
min_idx_flat = np.argmin(upper_vals)
min_p = upper_vals[min_idx_flat]
idx_pairs = np.column_stack(np.where(mask))
i, j = idx_pairs[min_idx_flat]

In [ ]:
S1, S2 = df.iloc[:, i], df.iloc[:, j]

Picking the best-ranked pair gives us a concrete example to carry through the workflow. Remember that the test is symmetric, but the tradable spread is not—next we should estimate a hedge ratio so the residual reflects the true linear relation rather than raw price levels. Treat this selection as a starting point to be confirmed out-of-sample.

## Standardize residuals and visualize signals
Reconfirm the cointegration result on the chosen pair, then build a spread and a rolling z-score to make signals comparable over time.

In [ ]:
score, pvalue, _ = coint(S1, S2)
print(
    f"tickers with lowest p-value: {tickers[i]} x {tickers[j]}, p={pvalue}"
)
spread = S1 - S2
zscore = (
    spread - spread.rolling(21, min_periods=21).mean()
) / spread.rolling(21, min_periods=21).std()

Standardizing with a 21-day rolling window avoids lookahead and puts the residual in “number of sigmas,” which makes entry and exit rules clean. The spread here is a simple difference; in practice, regress S1 on S2 and use residual = S1 − beta*S2 to respect scaling. Z-scores are portable across pairs and regimes, which helps when you later compare signals or batch-trade a basket.

Visualize the cointegration p-values, masking non-candidates and the redundant lower triangle to see which names truly co-move in a stable way.

In [ ]:
mask = (
    np.tril(np.ones_like(pvalue_matrix, dtype=bool))
    | (pvalue_matrix >= 0.10)
)
plt.figure(figsize=(8, 6))
sns.heatmap(
    pvalue_matrix,
    mask=mask,
    xticklabels=tickers,
    yticklabels=tickers,
    cmap="RdYlGn_r",
    annot=True,
    fmt=".2f",
    cbar=True,
    vmin=0,
    vmax=1,
)
plt.title("Cointegration Test p-value Matrix")
plt.show()

A heatmap helps us spot clusters of stable relationships within the renewable theme, which is more actionable than eyeballing charts. This is how we define a tradable subset before doing any signal tuning. Keep in mind that what looks good in one year may drift, so periodic re-tests are part of a robust workflow.

Plot the raw spread with simple bands and the standardized z-score so we can sketch an entry/exit plan around mean reversion.

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(14, 8), sharex=True)
axes[0].plot(spread.index, spread, label="Spread")
axes[0].axhline(spread.mean(), color="black", linestyle="--", lw=1)
axes[0].axhline(
    spread.mean() + spread.std(),
    color="red",
    linestyle="--",
    lw=1,
)
axes[0].axhline(
    spread.mean() - spread.std(),
    color="green",
    linestyle="--",
    lw=1,
)
axes[0].set_ylabel("Spread")
axes[0].legend()
axes[1].plot(zscore.index, zscore, label="Z-score")
axes[1].axhline(0, color="black", linestyle="--", lw=1)
axes[1].axhline(1, color="red", linestyle="--", lw=1)
axes[1].axhline(-1, color="green", linestyle="--", lw=1)
axes[1].set_ylabel("Z-score")
axes[1].legend()
plt.xlabel("Date")
plt.suptitle("Spread and Rolling Z-score between ABGB and FSLR")
plt.tight_layout()
plt.show()

Z-score bands make the rules explicit: for example, consider entries beyond ±1 and exits near 0, then evaluate with costs and borrow constraints. Treat these pictures as diagnostics; the next step is to backtest with rolling hedge-ratio estimation so bands reflect the true residual. Static thresholds are a useful baseline, but the edge lives in disciplined estimation and risk controls.

<a href="https://pyquantnews.com/">PyQuant News</a> is where finance practitioners level up with Python for quant finance, algorithmic trading, and market data analysis. Looking to get started? Check out the fastest growing, top-selling course to <a href="https://gettingstartedwithpythonforquantfinance.com/">get started with Python for quant finance</a>. For educational purposes. Not investment advice. Use at your own risk.